** What I want to do: ** Process test files for distracted driver and submit to kaggle

## Admin stuff

In [1]:
%matplotlib inline

In [2]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from datetime import datetime
import re

In [3]:
data_dir = "data"
train_path = "data/train/"
test_path = "data/test/"
validation_path = "data/valid/"
sample_train_path = "data/sample/train/"
sample_validation_path = "data/sample/valid/"
results_path = "data/results/"

In [4]:
# some useful utilities and the vgg16 model
import utils; reload(utils)
from utils import plots
from utils import save_array, load_array

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [5]:
# import the vgg16 model
import vgg16; reload(vgg16)
from vgg16 import Vgg16, image

## Load model weights


In [6]:
# instantiate the vgg16 model
vgg = Vgg16()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [7]:
Fork here. The trained model was fit on the test set once. The results are saved in "test_preds.dat" in the results path. For subsequent runs, simply load the results.

SyntaxError: invalid syntax (<ipython-input-7-eab88767a934>, line 1)

In [7]:
vgg.model.load_weights(results_path+'ft_1epoch.h5')

## Run model against test set

In [9]:
# for the vgg.test method to run, all images need to be part of another class.
# Lets call it 'unknown'
len(os.listdir(test_path))

79726

In [13]:
tempdir = test_path+'unknown'
!mkdir $tempdir

In [21]:
img_mv_counter = 0
for img in os.listdir(test_path):
    if os.path.isfile(os.path.join(test_path,img)):
        os.rename(os.path.join(test_path,img),os.path.join(test_path,'unknown',img))
        img_mv_counter+=1
print ("{} images moved to unknown folder.".format(str(img_mv_counter)))

17962 images moved to unknown folder.


In [22]:
# start logging time to execute
starttime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

test_batches, test_proba = vgg.test(test_path, batch_size = 64)

# log time for when the process ends
endtime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print ("timedelta = ", datetime.strptime(endtime,'%Y-%m-%d %H:%M:%S') - datetime.strptime(starttime,'%Y-%m-%d %H:%M:%S'))

Found 79726 images belonging to 1 classes.
timedelta =  0:32:13


## Build submission from predicted probabilities

In [23]:
# get filenames for each validation set
filenames = test_batches.filenames
filenames[:10]

['unknown/img_1.jpg',
 'unknown/img_10.jpg',
 'unknown/img_100.jpg',
 'unknown/img_1000.jpg',
 'unknown/img_100000.jpg',
 'unknown/img_100001.jpg',
 'unknown/img_100002.jpg',
 'unknown/img_100003.jpg',
 'unknown/img_100004.jpg',
 'unknown/img_100005.jpg']

In [25]:
len(filenames)

79726

In [26]:
test_proba.shape

(79726, 10)

In [27]:
# taking a look at the first 2...
test_proba[0:2,:]

array([[  6.0330e-03,   6.2223e-03,   2.8167e-03,   3.5212e-04,   6.0967e-04,   1.8071e-01,
          5.2517e-06,   6.0826e-04,   7.3298e-01,   6.9665e-02],
       [  9.9047e-01,   1.2519e-04,   1.5722e-05,   4.6743e-06,   4.2948e-03,   1.7472e-03,
          2.7635e-04,   3.4640e-06,   9.6088e-05,   2.9628e-03]], dtype=float32)

In [28]:
save_array(results_path + 'test_preds.dat', test_proba)
save_array(results_path + 'filenames.dat', filenames)

## Load predictions on the test set

In [8]:
filenames = load_array(results_path + 'filenames.dat')
filenames.shape

(79726,)

In [9]:
test_proba = load_array(results_path + 'test_preds.dat')
test_proba.shape

(79726, 10)

In [10]:
import pandas as pd

In [11]:
result_cols = ['c'+str(i) for i in np.arange(10)]
result_cols

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [12]:
df_results = pd.concat(
    [pd.DataFrame(filenames, columns=['img']),
    pd.DataFrame(test_proba,columns=result_cols)], axis=1
)
print (df_results.shape)
df_results.head()

(79726, 11)


,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,unknown/img_1.jpg,6.032987e-03,6.222323e-03,2.816721e-03,3.521241e-04,6.096693e-04,1.807119e-01,5.251669e-06,6.082557e-04,0.732976,0.069665
1,unknown/img_10.jpg,9.904737e-01,1.251865e-04,1.572199e-05,4.674284e-06,4.294755e-03,1.747155e-03,2.763456e-04,3.464006e-06,0.000096,0.002963
2,unknown/img_100.jpg,1.362209e-02,9.830527e-01,2.594683e-10,8.205414e-16,1.933659e-10,8.799974e-11,6.640742e-08,3.303141e-03,0.000020,0.000002
3,unknown/img_1000.jpg,8.803302e-11,5.876022e-10,4.455612e-05,7.126668e-08,1.381636e-03,2.014804e-06,1.270227e-06,8.009474e-08,0.998563,0.000008
4,unknown/img_100000.jpg,2.422421e-02,1.017842e-03,2.111686e-01,6.267425e-01,7.430664e-02,1.001487e-03,8.054106e-03,4.135343e-05,0.031458,0.021985


In [13]:
# removing the unknown tag from the image name
df_results['img'] = df_results.img.apply(lambda x:x.split('/')[1])
df_results.tail()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
79721,img_99994.jpg,9.150860e-03,1.104862e-02,1.918663e-01,1.971925e-03,4.547592e-02,1.099859e-02,1.609442e-02,3.194277e-01,6.627248e-02,3.276932e-01
79722,img_99995.jpg,1.926638e-05,7.765095e-08,5.091511e-16,9.999774e-01,3.428230e-08,1.160470e-11,2.879609e-17,6.307523e-09,8.505614e-11,3.275786e-06
79723,img_99996.jpg,1.373344e-02,4.532043e-01,6.449986e-07,2.434272e-01,2.230072e-01,1.594015e-04,1.990736e-03,1.383538e-05,3.485508e-02,2.960822e-02
79724,img_99998.jpg,1.248660e-10,5.004015e-06,9.995346e-01,1.618402e-12,1.926214e-10,1.667080e-12,4.538681e-04,4.540726e-10,6.531322e-06,1.450134e-12
79725,img_99999.jpg,1.253184e-03,1.387787e-04,2.255472e-13,9.734587e-09,1.757611e-11,9.983570e-01,4.224954e-09,2.509316e-04,7.473914e-08,5.212592e-11


In [14]:
# save to results
df_results.to_csv(results_path+'1epoch_submission1.csv',index=False)

In [15]:
from IPython.display import FileLink

In [16]:
FileLink(results_path+'1epoch_submission1.csv')

/home/ubuntu/distracted_driver/data/results/1epoch_submission1.csv

This resulted in an abyssmal score of 2.083 in private leaderboard, which was 1128 out of 1440 :|